# ID5059 Group Project: Temperature Prediction

### 2-1 Modelling: Neural Network

In [1]:
# import libraries
from pathlib import Path
import numpy as np
from tensorflow import keras
from sklearn.model_selection import GridSearchCV

RANDOM_STATE = 1
np.random.seed(RANDOM_STATE)

#### 2-1.1 Load Data

In [2]:
# function reads in data from file
def load_data(file_name, file_path = "./", deliniator = ","):
    path = Path() / file_path / file_name; print(path)
    if not path.is_file():
        raise Exception("File not found")

    return np.loadtxt(path, delimiter = deliniator)

# reads in csv file containing data
file_path = "./"

file_name = "x_train.csv"
x_train = load_data(file_name, file_path = file_path, deliniator = ",")

file_name = "y_train.csv"
y_train = load_data(file_name, file_path = file_path, deliniator = ",")

file_name = "x_valid.csv"
x_valid = load_data(file_name, file_path = file_path, deliniator = ",")

file_name = "y_valid.csv"
y_valid = load_data(file_name, file_path = file_path, deliniator = ",")

x_train.csv
y_train.csv
x_valid.csv
y_valid.csv


In [3]:
# checks to ensure that all of the data types are compatible for use in modelling
type(x_train) == type(x_valid) == type(y_train) == type(y_valid)

True

#### 2-1.2 Implement Neural Network

In [4]:
def R2(yTrue, yPredicted):
    residual = keras.backend.sum( (yTrue - yPredicted) ** 2 )
    total = keras.backend.sum( (yTrue - keras.backend.mean(yTrue)) ** 2 )
    return 1 - (residual / ( total + keras.backend.epsilon() ))

In [ ]:
def create_neural_network():
    model = keras.models.Sequential()
    model.add( keras.layers.Dense(24, input_shape = (x_train.shape[1], ), activation = "relu",
                                  kernel_regularizer = keras.regularizers.L1L2(l1 = 0.01 , l2 = 0.01))
                                  )
    model.add( keras.layers.Dense(16, activation = "relu",
                                  kernel_regularizer = keras.regularizers.L1L2(l1 = 0.01 , l2 = 0.01))
                                  )
    model.add( keras.layers.Dense(8, activation = "relu",
                                  kernel_regularizer = keras.regularizers.L1L2(l1 = 0.01 , l2 = 0.01))
                                  )
    model.add( keras.layers.Dense(1, activation = "linear") )

    model.compile(loss = "mse",
                  optimizer = keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07),
                  metrics = [keras.metrics.MeanSquaredError(), R2] )
    return model

In [7]:
neural_network = keras.wrappers.scikit_learn.KerasRegressor(build_fn = create_neural_network)

parameter_grid = {
    "batch_size": [16, 32],
    "epochs": [5, 10],
    "shuffle": [True, False],
    "verbose": [0]
}

k = 3
grid = GridSearchCV(estimator = neural_network,
                    param_grid = parameter_grid,
                    scoring = ["neg_mean_squared_error", "r2"],
                    refit = "neg_mean_squared_error", n_jobs = -1, cv = k, verbose = 3)


N0 = 0.1
N = int(N0 * x_train.shape[0])
grid.fit( np.random.choice(a = x_train, size = N, replace = False), np.random.choice(a = y_train, size = N, replace = False))

NameError: name 'create_neural_network' is not defined